In [ ]:
!pip install gradio PyPDF2 pyinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.1/710.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.7/344.7 kB 16.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from PyPDF2 import PdfMerger, PdfReader, PdfWriter
import os
import tempfile

def merge_pdfs(files):
    """Função para juntar PDFs"""
    if len(files) < 2:
        return None, "Por favor, selecione pelo menos 2 arquivos PDF para juntar."

    try:
        merger = PdfMerger()
        for file in files:
            merger.append(file.name)

        # Criar arquivo temporário para o resultado
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as temp_file:
            merger.write(temp_file.name)
            merger.close()
            return temp_file.name, "PDFs unidos com sucesso!"

    except Exception as e:
        return None, f"Erro ao unir PDFs: {str(e)}"

def split_pdf(file):
    """Função para dividir PDF em páginas individuais"""
    if file is None:
        return None, "Por favor, selecione um arquivo PDF para dividir."

    try:
        pdf = PdfReader(file.name)
        output_files = []

        # Criar diretório temporário para armazenar as páginas
        temp_dir = tempfile.mkdtemp()

        for page_num in range(len(pdf.pages)):
            writer = PdfWriter()
            writer.add_page(pdf.pages[page_num])

            output_path = os.path.join(temp_dir, f"pagina_{page_num + 1}.pdf")
            with open(output_path, "wb") as output_file:
                writer.write(output_file)
            output_files.append(output_path)

        return output_files, "PDF dividido com sucesso!"

    except Exception as e:
        return None, f"Erro ao dividir PDF: {str(e)}"

# Criar interface Gradio
with gr.Blocks(title="Manipulador de PDF") as demo:
    gr.Markdown("# Manipulador de PDF")

    with gr.Tab("Juntar PDFs"):
        with gr.Column():
            files_input = gr.File(
                file_count="multiple",
                file_types=[".pdf"],
                label="Selecione os arquivos PDF para juntar"
            )
            merge_button = gr.Button("Juntar PDFs")
            merge_output = gr.File(label="PDF Unificado")
            merge_message = gr.Textbox(label="Status", interactive=False)

        merge_button.click(
            fn=merge_pdfs,
            inputs=[files_input],
            outputs=[merge_output, merge_message]
        )

    with gr.Tab("Dividir PDF"):
        with gr.Column():
            file_input = gr.File(
                file_types=[".pdf"],
                label="Selecione um arquivo PDF para dividir"
            )
            split_button = gr.Button("Dividir PDF")
            split_output = gr.File(
                file_count="multiple",
                label="Páginas Individuais"
            )
            split_message = gr.Textbox(label="Status", interactive=False)

        split_button.click(
            fn=split_pdf,
            inputs=[file_input],
            outputs=[split_output, split_message]
        )

# Iniciar a aplicação
if __name__ == "__main__":
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19115ae319786481da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Requisitos do Manipulador de PDF

## 1. Requisitos Funcionais (RF)

### RF01 - Upload de Arquivos
- O sistema deve permitir ao usuário fazer upload de múltiplos arquivos PDF
- O sistema deve validar se os arquivos enviados são do formato PDF
- O sistema deve exibir mensagem de erro caso o arquivo não seja PDF

### RF02 - Junção de PDFs
- O sistema deve permitir a junção de dois ou mais arquivos PDF
- O sistema deve manter a ordem dos arquivos conforme selecionado pelo usuário
- O sistema deve gerar um único arquivo PDF como resultado da junção
- O sistema deve permitir o download do arquivo PDF resultante

### RF03 - Divisão de PDF
- O sistema deve permitir a divisão de um arquivo PDF em múltiplas páginas individuais
- O sistema deve gerar um arquivo PDF separado para cada página do documento original
- O sistema deve nomear os arquivos resultantes de forma sequencial (página_1.pdf, página_2.pdf, etc.)
- O sistema deve permitir o download dos arquivos PDF resultantes

### RF04 - Feedback ao Usuário
- O sistema deve exibir mensagens de sucesso após operações bem-sucedidas
- O sistema deve exibir mensagens de erro em caso de falhas
- O sistema deve informar o progresso das operações em andamento

## 2. Requisitos Não Funcionais (RNF)

### RNF01 - Usabilidade
- A interface deve ser intuitiva e fácil de usar
- O sistema deve ser acessível via navegador web
- O sistema deve suportar operação de arrastar e soltar (drag and drop) para upload de arquivos
- A interface deve ser responsiva e adaptável a diferentes tamanhos de tela

### RNF02 - Desempenho
- O sistema deve processar a junção de PDFs em até 30 segundos para arquivos de até 20MB
- O sistema deve processar a divisão de PDF em até 20 segundos para arquivos de até 10MB
- O sistema deve suportar o upload simultâneo de até 10 arquivos
- O tempo de resposta da interface não deve exceder 2 segundos

### RNF03 - Segurança
- O sistema deve processar os arquivos em ambiente isolado
- Os arquivos temporários devem ser excluídos após o processamento
- O sistema não deve armazenar permanentemente os PDFs processados
- O sistema deve garantir que apenas arquivos PDF sejam processados

### RNF04 - Disponibilidade
- O sistema deve estar disponível para uso local sempre que iniciado
- O sistema deve manter a sessão ativa durante todo o processamento
- O sistema deve ser capaz de se recuperar de falhas sem perder dados

### RNF05 - Portabilidade
- O sistema deve funcionar nos principais navegadores (Chrome, Firefox, Safari, Edge)
- O sistema deve ser executável em diferentes sistemas operacionais (Windows, macOS, Linux)
- O sistema deve requerer instalação mínima de dependências

### RNF06 - Manutenibilidade
- O código deve ser modular e bem documentado
- O sistema deve permitir fácil atualização das dependências
- O sistema deve gerar logs de erros para facilitar a depuração

### RNF07 - Escalabilidade
- O sistema deve suportar o processamento de PDFs de diferentes tamanhos
- O sistema deve gerenciar eficientemente o uso de memória durante o processamento
- O sistema deve ser capaz de lidar com múltiplas requisições simultâneas

### RNF08 - Conformidade
- O sistema deve preservar a qualidade original dos PDFs processados
- O sistema deve manter a formatação original dos documentos
- O sistema deve suportar diferentes versões do formato PDF

In [ ]:
!pip uninstall pathlib -y

Found existing installation: pathlib 1.0.1
Uninstalling pathlib-1.0.1:
  Successfully uninstalled pathlib-1.0.1


SyntaxError: invalid syntax (<ipython-input-12-707e325a8bb6>, line 36)